# Vocab Analysis
## Section 3: Analyze the Data

### 1. Import necessary libraries

In [1]:
import pandas as pd

### 2. Import necessary datasets

In [ ]:
# import notes


# import cards


# import combo



In [ ]:
# Observe Metadata (tag) Frequency:

In [ ]:
tag_freq = pd.Series(' '.join(df_notes_final.tags).split()).value_counts()

tag_freq.head(20)

# Initial Observations

Looks like our data is ready for some proper inspection! What are some questions that we might ask of this dataset? We could start with some simple/basic broad/overview observations about the (condensed) dataset such as:
- How many terms (unique notes) exist?
- How many study vectors (unique card types) exist (were utilized by student A)?
- When did student A first start studying?
- What is the data distribution for reps count? For laspes count?
- Of the terms that exist, how many had audio data?
- Of the terms that exist, how many had image data?

In [ ]:
# unique terms in the condensed dataset
len(df_new_cols['Term'].unique())

# confirm what card types exist
df_new_cols['ord'].value_counts()

pd_crt # datetime of collection creation (studying commenced from this date)

print(df_new_cols.shape)

# Intitial Analysis

There appears to be a linear relationship between lapses & reps. This seems to make sense and is worth keeping in mind (that lapses would, it seems, incur a cost of increasing reps). However, this info doesn't seem (to the author) directly actionable, whether it be simple correlation or even causation. The primary focus is what can be done to optimize studying.

# Topical Analysis

After doing some basic assessments of the data, we can dig a bit deeper:
- Is there a correlation between words having multiple readings ("yomi") and their forget rate\*?
- Is there a correlation between words having same/similar sounding words and their forget rate\*?
- What might the effect of word length be on memorability? \*\*, \*\*\*

> \* Forget rate can be understand as a multitude of things, such as the ratio between lapses & reps, as well as the raw lapse count, the average interval, and other numbers/ratios to be determined. I will attempt to clarify this in the process.  
\*\* Memorability being loosely correlated with forget rate, where memorability could be understood as a word/term's intrinsic "stickiness" in the brain, as opposed to an individual or collective's capacity to keep words/terms in their head. Sources pending.  
\*\*\* A huge caveat here being that, this dataset has a sample size of 1 (for both student and language), so all observations, interpretations, and understandings must be taken with more than a few grains of salt (and tested further with larger sample sizes, of at least 200 students, and 5 or more languages).

In [ ]:
#show correlation of stats via heatmap
df_worked = df_binary2.copy() # 'ivl','factor','lapses'
graph_drop_cols_1 = [
    'nid','commonword','clothing','animal','body','food','textbook','college','place',
    'fromdict','fromexam','onechar','n1','n2','n3','n4','n5','hasVisual',
    'hasAudio','hasSimilar','hasAltForm','TermLen','Syllables','ivl_q']
df_worked = df_worked.drop(graph_drop_cols_1,axis=1)
corr = df_worked.corr()
# https://stackoverflow.com/questions/38913965/make-the-size-of-a-heatmap-bigger-with-seaborn
fig, ax = plt.subplots(figsize=(10,10))         # Sample figsize in inches
sns.heatmap(corr, vmin=-1, annot=True)

In [ ]:
# todo: move this up one cell at least, if not up to the very top of initial graphing
# Basic correlogram
sns.pairplot(df_explore)
plt.show()

# Further Analysis

For a deeper understanding of what it means to aquire new terminology, the researcher believes it best to conduct analysis on term acquisition by merging multiple vectors (individual cards) of a single term into single entries, where dummy values for each vector (such as review count, lapse count, etc.) are encoded per entry. This would enable inspection and correlation analysis of:
- total reviews per term
- average ratio of reviews per term per vector (look vs hear vs recall vs read)
- where lapses are most likely to occur (per word, per vector, etc.)
- how word length, presence of kanji, katakana, hirgana, or combination thereof, may affect the above counts & ratios

# Further Information

The Spaced Repetition Software (\"SRS\") used for the study of Japanese by student \"A\" is an open souce program called Anki. The algorithm used by it to \"graduate\" (also refered to as \"maturing\") study items (called cards) so that subsequent reviews/practices will be spaced into the future is referred to as SM-2. [Please click here for more information on the SM-2 algorithm used in Anki.]("https://apps.ankiweb.net/docs/manual.html#what-algorithm")